In [80]:
# ---------------------------------------------------------------------------- #
#                               Import Statements                              #
# ---------------------------------------------------------------------------- #

# for parsing
from bs4 import BeautifulSoup
import lxml

# for making requests and scraping the html
import requests

# for database management
import sqlite3

# for multithreading
import concurrent.futures

# for directory paths
import os

# for arguments
import sys
import datetime

# for writing into csv files and saving the data
import pandas as pd

# For utlitly
import sys

# For the spinners
from Spinner.spinner import Spinner
# from Spinner.spinnerEasy import Spinner


# ---------------------------------------------------------------------------- #
#                               Utility Functions                              #
# ---------------------------------------------------------------------------- #

def urlJoin(service, attach):
    """
    Joins the urls
    """

    if service[len(service)-1] == '/':
        service = service[:len(service)-1]
    if attach[0] == '/':
        attach = attach[1:]
    joined = service+'/'+attach
    return joined


def get_HTML(url):
    """
    Utility Function that takes in the URL and returns the response's html directly if the status code is 200
    """

    response = requests.get(url)

    if response.status_code == 200:
        html_response = response.content
        return html_response

    return ""


def get_user(username):

    user_url = f"https://codeforces.com/profile/{username}"
    return user_url


In [81]:
def get_info_img(id):
    
    url = get_user(id)
    html = get_HTML(url)
    soup = BeautifulSoup(html)
    try :
        userbox = soup.find('div', class_='userbox')
        title_photo = userbox.find('div',class_='title-photo')
        info = userbox.find('div',class_='info')
    except:
        title_photo=None
        info=None
    return [info, title_photo]

In [82]:
def get_information_dict(info):
    main_info = info.find('div',class_='main-info')
    ul = info.find('ul')
    lis = ul.findAll('li')
    lis_text = []
    for li in lis:
        lis_text.append(li.text.strip())
    lis_text=lis_text[:-2]
    information = {
        'rating': 0,
        'contribution': 0,
        'friends': 0,
        'lastseen': '',
        'registered': ''
    }
    for li in lis_text:
        temp = li.split(':')
        temp[0] = temp[0].strip()
        temp[1] = temp[1].strip()
        if temp[0]=='Contest rating':
            temp[0]='rating'
            temp[1]=int(temp[1][:4])
        if temp[0] =='Contribution':
            temp[0]='contribution'
            temp[1]=int(temp[1])
        if temp[0]=='Friend of':
            temp[0]='friends'
            temp[1]=int(temp[1].split(' ')[0])
        if temp[0]=='Last visit':
            temp[0]='lastseen'
        if temp[0]=='Registered':
            temp[0]='registered'

        information[temp[0]] = temp[1]
    return information


In [83]:
def get_img_bytes(title_photo):
    
    profile_pic = requests.get('https:'+title_photo.find('img')["src"]).content
    return profile_pic

In [84]:
[info, title_photo] = get_info_img('just_me_007')

In [87]:
img_bytes = get_img_bytes(title_photo)
with open('pp.jpg','wb') as fp:
    fp.write(img_bytes)

In [90]:
from pprint import pprint

information = get_information_dict(info)
pprint(information)

{'contribution': 0,
 'friends': 129,
 'lastseen': 'online now',
 'rating': 1449,
 'registered': '2 months ago'}
